In [1]:
#!wget https://uaelxg.bn.files.1drv.com/y4mfVVREWQ1x_HQo_XjcLkz4Ae0EAdMbms2x6o5FQfnwhCpvZmgYAZsRA0A8UIeISKnECDolIAbdclP-YkAN12KLPImrtIQ4vbJquc_Joty_5Ouvb5gNZ1HY5vhnb-EaXl3P7sBVZk8j_-L3QhGHiPhUXEYAyGoMf0s3TnYYKh_hIxL-oz_hFShHbBrNUJu2yLQ/fruits-360_dataset_2018_06_03.zip
  
 

--2018-06-23 03:33:04--  https://uaelxg.bn.files.1drv.com/y4mfVVREWQ1x_HQo_XjcLkz4Ae0EAdMbms2x6o5FQfnwhCpvZmgYAZsRA0A8UIeISKnECDolIAbdclP-YkAN12KLPImrtIQ4vbJquc_Joty_5Ouvb5gNZ1HY5vhnb-EaXl3P7sBVZk8j_-L3QhGHiPhUXEYAyGoMf0s3TnYYKh_hIxL-oz_hFShHbBrNUJu2yLQ/fruits-360_dataset_2018_06_03.zip
Resolving uaelxg.bn.files.1drv.com (uaelxg.bn.files.1drv.com)... 204.79.197.213
Connecting to uaelxg.bn.files.1drv.com (uaelxg.bn.files.1drv.com)|204.79.197.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211922343 (202M) [application/zip]
Saving to: ‘fruits-360_dataset_2018_06_03.zip’

fruits-360_dataset_ 100%[===================>] 202.10M  21.1MB/s    in 10s     

2018-06-23 03:33:15 (19.7 MB/s) - ‘fruits-360_dataset_2018_06_03.zip’ saved [211922343/211922343]



In [0]:
# import zipfile
# zip_ref = zipfile.ZipFile('fruits-360_dataset_2018_06_03.zip', 'r')
# zip_ref.extractall('data2')
# zip_ref.close()

In [12]:
!ls

data2  datalab	fruits-360_dataset_2018_06_03.zip


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import glob
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [0]:
#Load dữ liệu tập train và test

In [0]:
training_fruit_img = []
training_label = []
for dir_path in glob.glob("data2/fruits-360/Training/*"):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        training_fruit_img.append(image)
        training_label.append(img_label)
training_fruit_img = np.array(training_fruit_img)
training_label = np.array(training_label)
label_to_id = {v:k for k,v in enumerate(np.unique(training_label)) }
id_to_label = {v:k for k,v in label_to_id.items()}
training_label_id = np.array([label_to_id[i] for i in training_label]) # chuyển trainlabel từ tên trái cây thành chỉ số từ 0->64
validation_fruit_img=[]
validation_label =[]
for dir_path in glob.glob("data2/fruits-360/Validation/*"):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        validation_fruit_img.append(image)
        validation_label.append(img_label)
validation_fruit_img = np.array(validation_fruit_img)
validation_label = np.array(validation_label)
validation_label_id = np.array([label_to_id[i] for i in validation_label]) # chuyển validationlabel từ tên trái cây thành chỉ số từ 0->64

In [27]:
#tạo cnn model

batch_size = 80
num_classes = 65
epochs = 40

# input image dimensions
img_rows, img_cols = 64, 64
input_shape = (img_rows,img_cols,3)


y_train = training_label_id
y_test = validation_label_id

x_train = training_fruit_img.astype('float32')
x_test = validation_fruit_img.astype('float32')

# chuyển các giá trị dữ liệu hình ảnh thành 0->1
x_test /= 255
x_train /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#tạo checkpoint để lưu lại model tại epoch có giá trị val_acc cao nhất
checkpoint = keras.callbacks.ModelCheckpoint(filepath = "weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_acc', verbose=0, save_best_only=True,
                                             save_weights_only=False, mode='max', period=1)
model = Sequential()

model.add(Conv2D(32, kernel_size= (3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size= (3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer = keras.optimizers.RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0),
             metrics=['accuracy'])
model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, y_test),
         callbacks = [checkpoint])
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

from sklearn.metrics import classification_report
classes = model.predict_classes(x_test, batch_size = None)
print(classification_report(validation_label_id, classes))


x_train shape: (32426, 64, 64, 3)
32426 train samples
10903 test samples
Train on 32426 samples, validate on 10903 samples
Epoch 1/40
32426/32426 [==============================] - 29s 888us/step - loss: 0.9091 - acc: 0.7460 - val_loss: 0.3203 - val_acc: 0.8937
Epoch 2/40
 2480/32426 [=>............................] - ETA: 22s - loss: 0.1736 - acc: 0.9419

32426/32426 [==============================] - 28s 858us/step - loss: 0.1074 - acc: 0.9662 - val_loss: 0.1874 - val_acc: 0.9469
Epoch 3/40
24480/32426 [=====================>........] - ETA: 6s - loss: 0.0601 - acc: 0.9801

32426/32426 [==============================] - 28s 858us/step - loss: 0.0571 - acc: 0.9810 - val_loss: 0.2024 - val_acc: 0.9428
Epoch 4/40
32426/32426 [==============================] - 28s 854us/step - loss: 0.0377 - acc: 0.9860 - val_loss: 0.1731 - val_acc: 0.9529
Epoch 5/40
  240/32426 [..............................] - ETA: 24s - loss: 0.0065 - acc: 1.0000

32426/32426 [==============================] - 28s 859us/step - loss: 0.0324 - acc: 0.9883 - val_loss: 0.1776 - val_acc: 0.9575
Epoch 6/40
23680/32426 [====================>.........] - ETA: 6s - loss: 0.0248 - acc: 0.9891

32426/32426 [==============================] - 28s 853us/step - loss: 0.0241 - acc: 0.9897 - val_loss: 0.1863 - val_acc: 0.9589
Epoch 7/40
32426/32426 [==============================] - 28s 855us/step - loss: 0.0261 - acc: 0.9899 - val_loss: 0.1763 - val_acc: 0.9590


...

32426/32426 [==============================] - 27s 847us/step - loss: 0.0155 - acc: 0.9922 - val_loss: 0.3157 - val_acc: 0.9590
Epoch 35/40
24240/32426 [=====================>........] - ETA: 6s - loss: 0.0163 - acc: 0.9917

32426/32426 [==============================] - 27s 846us/step - loss: 0.0170 - acc: 0.9920 - val_loss: 0.9073 - val_acc: 0.9074
Epoch 36/40
32426/32426 [==============================] - 27s 847us/step - loss: 0.0174 - acc: 0.9921 - val_loss: 0.3039 - val_acc: 0.9664
Epoch 37/40
   80/32426 [..............................] - ETA: 25s - loss: 0.0066 - acc: 1.0000

32426/32426 [==============================] - 28s 848us/step - loss: 0.0192 - acc: 0.9919 - val_loss: 0.4288 - val_acc: 0.9564
Epoch 38/40
23520/32426 [====================>.........] - ETA: 6s - loss: 0.0167 - acc: 0.9917

32426/32426 [==============================] - 28s 860us/step - loss: 0.0183 - acc: 0.9916 - val_loss: 0.3124 - val_acc: 0.9655
Epoch 39/40
32160/32426 [============================>.] - ETA: 0s - loss: 0.0202 - acc: 0.9918

32426/32426 [==============================] - 27s 844us/step - loss: 0.0202 - acc: 0.9918 - val_loss: 0.4002 - val_acc: 0.9561
Epoch 40/40
32426/32426 [==============================] - 27s 836us/step - loss: 0.0223 - acc: 0.9915 - val_loss: 0.4137 - val_acc: 0.9560
Test loss: 0.41371132182068915
Test accuracy: 0.9559754196092819
             precision    recall  f1-score   support

          0       0.80      0.98      0.88       164
          1       1.00      0.99      1.00       164
          2       0.90      1.00      0.95       164
          3       0.67      1.00      0.81       161
          4       0.89      0.85      0.87       164
          5       1.00      0.81      0.90       164
          6       1.00      0.72      0.84       164
          7       0.94      1.00      0.97       144
          8       1.00      1.00      1.00       166
          9       1.00      1.00      1.00       164
         10       1.00      1.00      1.00       164
         11       1.00      1.

In [0]:
#tại epochs = 14 đạt giá trị val_acc = 0.97 cao nhất: load model tại epoch = 27:



In [28]:
!ls




data2				   weights.01-0.36.hdf5  weights.07-0.18.hdf5
datalab				   weights.02-0.19.hdf5  weights.08-0.14.hdf5
fruits-360_dataset_2018_06_03.zip  weights.03-0.17.hdf5  weights.10-0.15.hdf5
models				   weights.04-0.17.hdf5  weights.14-0.11.hdf5
store				   weights.05-0.18.hdf5
weights.01-0.32.hdf5		   weights.06-0.19.hdf5


In [29]:
modelbest = keras.models.load_model('weights.14-0.11.hdf5')
from sklearn.metrics import classification_report
classes = modelbest.predict_classes(x_test, batch_size = None)
print(classification_report(validation_label_id, classes))

             precision    recall  f1-score   support

          0       0.80      0.84      0.82       164
          1       1.00      0.99      1.00       164
          2       1.00      1.00      1.00       164
          3       0.80      0.99      0.88       161
          4       0.99      0.96      0.98       164
          5       1.00      0.99      1.00       164
          6       1.00      0.88      0.94       164
          7       1.00      1.00      1.00       144
          8       1.00      1.00      1.00       166
          9       1.00      1.00      1.00       164
         10       1.00      1.00      1.00       164
         11       1.00      1.00      1.00       143
         12       0.99      1.00      0.99       166
         13       1.00      0.93      0.96       166
         14       0.93      0.95      0.94       166
         15       0.95      1.00      0.97       166
         16       1.00      1.00      1.00       164
         17       0.83      0.99      0.90   

In [31]:
modelbest.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 26, 26, 64)        18496     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 13, 13, 64)        0         
__________

In [19]:
# sử dụng randomforest để phân loại

rf = RandomForestClassifier(n_estimators=100)
rf.fit(training_fruit_img.reshape(training_fruit_img.shape[0],64*64*3), training_label_id)
pred = rf.predict(validation_fruit_img.reshape(validation_fruit_img.shape[0],64*64*3))


from sklearn.metrics import classification_report
print(classification_report(y_test, pred))
print(np.sum(y_test==pred))

             precision    recall  f1-score   support

          0       0.76      0.71      0.74       164
          1       0.77      0.94      0.85       164
          2       0.94      1.00      0.97       164
          3       1.00      1.00      1.00       161
          4       1.00      1.00      1.00       164
          5       1.00      1.00      1.00       164
          6       0.93      1.00      0.96       164
          7       1.00      1.00      1.00       144
          8       1.00      1.00      1.00       166
          9       0.99      0.76      0.86       164
         10       1.00      1.00      1.00       164
         11       1.00      0.88      0.94       143
         12       1.00      1.00      1.00       166
         13       0.99      0.80      0.88       166
         14       0.80      0.84      0.82       166
         15       0.97      0.90      0.94       166
         16       0.99      1.00      1.00       164
         17       0.97      0.99      0.98   

In [0]:
#model dùng randomforest cho kết quả khá giống với cnn mặc dù accurancy thấp hơn.

apple.jpg			     lemon1.jpg  thahlongtest.jpg
data2				     lemon.jpg	 tmp
datalab				     logs	 vai.jpg
fruits-360_dataset_2018_06_03.zip    logs2
fruits-360_dataset_2018_06_03.zip.1  logs2.zip
